In [1]:
import telebot
import pandas as pd
import re
import random
from pycoingecko import CoinGeckoAPI
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import time

In [26]:
def call_coin_price(crypto,currency='usd'):
	# global id_
	id_ = 'none'
	cg = CoinGeckoAPI()
# 	print(crypto)
	list_coin = cg.get_coins_list()
	symbol = crypto
	if crypto == 'iota':
		symbol = 'miota'
	else:
		symbol = crypto
	# symbol = crypto
	for i, list_sym in enumerate(list_coin):
		if(symbol == list_sym['symbol'].lower()) and i >0:
# 			print('i: ',i)
			id_ = list_coin[i]['id']
	currency = currency
	if id_ == 'none':
		return 'có cái loz'
# 	print(id_)
	price = cg.get_price(ids=id_, vs_currencies=currency)
	result = price[id_][currency]
	return result

In [3]:
def num_finance(number,crypto):
	if crypto == 'btc':
		num_digit = 6
	elif crypto == 'none':
		num_digit = 4
	else:
		num_digit = 2

	if number == '':
		result = '-'
	elif float(number) <0:
		result = str('({:,.{}f})').format(abs(float(number)),num_digit)
	elif float(number) == 0:
		result = '-'
	else:
		result = str('{:,.{}f}').format(float(number),num_digit)
	return result

### Xu ly column

In [43]:
def rename_column(data,col_list):
    for col in col_list: 
        df.rename(columns={col: str(col).strip().lower()},inplace = True)

In [44]:
rename_column(data = df,col_list = df.columns)

In [45]:
df.columns

Index(['name', 'exchange', 'ccy', 'remaining_qty', 'current_avg_price',
       'holding_value', 'buy_quantity', 'avg_purchasing_price', 'total_cost',
       'sell_quantity', 'avg_selling_price', 'total_sales', 'pnl', 'pnl %'],
      dtype='object')

In [5]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope) # Your json file here

gc = gspread.authorize(credentials)

wks = gc.open_by_key('1W2_tdPIPJ8rNPMi4eYcN8NOzq2pEqM_83n85CT3Nqxo').worksheet("summary")

In [7]:
data = wks.get_all_values()

In [9]:
headers = data.pop(0)

In [10]:
df = pd.DataFrame(data, columns=headers)

In [94]:
holder = 'K'
df[(df['name'] == holder) & (df['remaining_qty'].astype('float64') >0 )]
list_coin = df[(df['name'] == holder) & (df['remaining_qty'].astype('float64') >0 )]['ccy'].tolist()
pnl = df[(df['name'] == holder) & (df['remaining_qty'].astype('float64') >0 )]['pnl'].tolist()
remain_qty = df[(df['name'] == holder) & (df['remaining_qty'].astype('float64') >0 )]['remaining_qty'].astype('float64').tolist()
list_coin

['BTC', 'DIA', 'POND', 'EKT', 'FILDA', 'JULD', 'NEST', 'ZKS']

- market_value = float(remain_qty) * float(market_price)
- net_position = market_value + float(pnl)

In [62]:
pnl

['2.4608',
 '-50.0486',
 '-50.1080',
 '-20.0400',
 '-70.1335',
 '-10.0180',
 '-8.0156',
 '-100.1595']

In [60]:
list_coin.tolist()

['BTC', 'DIA', 'POND', 'EKT', 'FILDA', 'JULD', 'NEST', 'ZKS']

In [66]:
len(lis)

'BTC'

In [77]:
remain_qty[0]

'0.000002'

In [139]:
list_price = []
list_net_position = []
total_net_position = 0
for i in range(0,len(list_coin)):
    crypto = list_coin[i]
    market_price = call_coin_price(str(crypto).lower())
#     list_price.append(price)
    print('----',i,'----')
    print(market_price)
    print(remain_qty[i])
    market_value = float(remain_qty[i])*market_price
    net_position = market_value+float(pnl[i])
    
    print(market_value,'---',net_position)
    list_net_position.append(net_position)
    total_net_position += net_position
    content += '''{} --- {}   --- $ {} \n'''.format(crypto,num_finance(remain_qty[i],crypto=crypto),num_finance(net_position,crypto = crypto))
list_price
total_net_position

---- 0 ----
56997
2e-06
0.113994 --- 2.574794
---- 1 ----
2.46
20.955
51.549299999999995 --- 1.5006999999999948
---- 2 ----
0.148271
332.39
49.28379768999999 --- -0.824202310000004
---- 3 ----
0.00543896
3496.5
19.017323639999997 --- -1.022676360000002
---- 4 ----
0.452597
134.36
60.81093292000001 --- -9.322567079999985
---- 5 ----
0.278878
36.12
10.07307336 --- 0.05507335999999974
---- 6 ----
0.03804346
201.96
7.6832571816 --- -0.332342818399999
---- 7 ----
2.55
37.74
96.237 --- -3.9224999999999994


-11.293721208399994

In [141]:
print(content)

ZKS --- 37.74   --- $ (3.92) 
BTC --- 0.00   --- $ 2.57 
DIA --- 20.95   --- $ 1.50 
POND --- 332.39   --- $ (0.82) 
EKT --- 3,496.50   --- $ (1.02) 
FILDA --- 134.36   --- $ (9.32) 
JULD --- 36.12   --- $ 0.06 
NEST --- 201.96   --- $ (0.33) 
ZKS --- 37.74   --- $ (3.92) 



In [104]:
list_coin
remain_qty
list_net_position

[2.5752919999999997,
 1.919799999999995,
 -0.012838319999993075,
 -1.000578479999998,
 -9.169934119999994,
 0.07374739999999846,
 -0.5231223127999991,
 -3.545099999999991]

In [126]:
text = '''<b>Shark K - Portfolio</b>\n
CCY --- remain_qty --- Net_Position\n'''

In [135]:
content = '''{} --- {}   --- $ {} \n'''.format(list_coin[0],num_finance(remain_qty[0],crypto='btc'),num_finance(list_net_position[0],crypto = 'none'))

In [136]:
content

'BTC --- 0.000002   --- $ 2.5753 \n'

In [137]:
print(text+content)

<b>Shark K - Portfolio</b>

CCY --- remain_qty --- Net_Position
BTC --- 0.000002   --- $ 2.5753 



In [109]:
text

'<b>Shark K - Portfolio</b>\n'

In [101]:
sum(list_net_position)

-10.11967761739999

In [96]:
remain_qty

2e-06

In [20]:
call_coin_price(crypto = 'btc')

btc
i:  782
bitcoin


57069